# Challenge Problems

## Q1: Worst Walk on Campus

**Note: this is for people who already know the Python basics.**

Some "refresher" documentation:
- https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions
- https://docs.python.org/2/library/itertools.html#itertools.combinations
- https://docs.python.org/2/library/functions.html

In [1]:
from geopy.geocoders import Nominatim

In [2]:
geolocator = Nominatim()

In [3]:
building_names = []
with open('buildings.txt', 'r') as f:
    for line in f.readlines():
        building_names.append(line.strip())

In [5]:
names_and_coordinates = {}
for i, building_name in enumerate(building_names):
    location = geolocator.geocode(building_name)
    if location and 'Berkeley' in location.address:
        names_and_coordinates[building_name] = (location.latitude, location.longitude)
    print('Halls scanned: {0}/{1}'.format(i + 1, len(building_names)), end='\r')

In [7]:
from itertools import combinations

In [21]:
from math import sin, cos, sqrt, atan2, radians
def haversine(lat1, lon1, lat2, lon2):
    """Calculates the straight line distance between two latitude-longitude coordinates"""
    # approximate radius of earth in miles
    R = 3959.0

    lat1_r = radians(lat1)
    lon1_r = radians(lon1)
    lat2_r = radians(lat2)
    lon2_r = radians(lon2)

    dlon = lon2_r - lon1_r
    dlat = lat2_r - lat1_r

    a = sin(dlat / 2)**2 + cos(lat1_r) * cos(lat2_r) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

You are trying to create the most evil schedule on campus, which contains back-to-back classes in buildings that are the furthest apart on campus. You are given the buildings and coordinates of those locations in a dictionary that can be queried as follows:

In [23]:
names_and_coordinates['Soda Hall'], names_and_coordinates['Cory Hall']

((37.8756741, -122.258724), (37.8750816, -122.257557811467))

Write a function that returns a tuple of the furthest two buildings on campus, and the distance in miles between the two.

In [16]:
# Solution
def furthest_walk_on_campus(names_and_coordinates):
    def key_fn(entry):
        (bname1, (lat1, lon1)), (bname2, (lat2, lon2)) = entry
        return haversine(lat1, lon1, lat2, lon2)
    (bname1, (lat1, lon1)), (bname2, (lat2, lon2)) = max(
        combinations(
            names_and_coordinates.items(),
            2
        ),
        key=key_fn
    )
    return bname1, bname2, haversine(lat1, lon1, lat2, lon2)

In [20]:
print('The furthest walk on campus is {0} to {1}, which is {2} miles'.format(*furthest_walk_on_campus(names_and_coordinates)))

The furthest walk on campus is Koshland Hall to Simon Hall, which is 0.7209184926896222 miles


**Extension**: Distance isn't always the best measure of the worst walk. Use Google Maps Matrix API (https://developers.google.com/maps/documentation/distance-matrix/intro)'s time to destination function to rank the walks instead.

## Q2: Solving Sudoku

http://lipas.uwasa.fi/~timan/sudoku/

In [35]:
def read_board(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        board = []
        for line in lines:
            board.append(line.split(' ')[:-1])
    return board[:-1]

In [ ]:
def check_solution(board):
    """Checks that the board has a valid solution based on Sudoku rules."""
    

In [27]:
def get_first_empty_cell(board):
    """Gets the first empty cell on the board, left to right, up to down"""
    for i, row in enumerate(board):
        for j, col in enumerate(row):
            if board[i][j] == '0':
                return i, j
    # if none available, return -1, -1
    return -1, -1

In [28]:
def is_placement_valid(board, i, j, val):
    """Checks if placing <val> at position (i, j) on the board is valid"""
    row_count = sum(1 for k in range(9) if board[k][j] == val)
    col_count = sum(1 for l in range(9) if board[i][l] == val)
    if row_count + col_count > 0:
        return False
    box_start_i, box_start_j = i - (i % 3), j - (j % 3)
    box_count = 0
    for i in range(3):
        for j in range(3):
            if board[box_start_i + i][box_start_j + j] == val:
                box_count += 1
    return box_count == 0

In [32]:
def solve(board):
    """Solves a Sudoku board: here are the steps:
    
        1. Find an empty cell
        2. Find a value to place in that empty cell, if none exist, return False
        3. Place the value in the empty cell
        4. Recursively call solve(board):
            a. If a solution is found, return it
            b. If no solution is found, return False
    """
    i, j = get_first_empty_cell(board)
    if i == -1:
        return True
    for p in range(1, 10):
        if is_placement_valid(board, i, j, str(p)):
            board[i][j] = str(p)
            if solve(board):
                return board
            board[i][j] = '0'
    return False

In [34]:
solve(read_board('sudoku-puzzles/puzzle1.txt'))

[['2', '9', '4', '8', '6', '3', '5', '1', '7'],
 ['7', '1', '5', '4', '2', '9', '6', '3', '8'],
 ['8', '6', '3', '7', '5', '1', '4', '9', '2'],
 ['1', '5', '2', '9', '4', '7', '8', '6', '3'],
 ['4', '7', '9', '3', '8', '6', '2', '5', '1'],
 ['6', '3', '8', '5', '1', '2', '9', '7', '4'],
 ['9', '8', '6', '1', '3', '4', '7', '2', '5'],
 ['5', '2', '1', '6', '7', '8', '3', '4', '9'],
 ['3', '4', '7', '2', '9', '5', '1', '8', '6']]